: 

In [3]:
import nfl_func
roster_columns = ['team', 'position', 'depth_chart_position', 'jersey_number','status', 'player_name', 'first_name', 'last_name', 'birth_date', 'height', 'weight', 'player_id', 'years_exp']
roster_data = nfl_func.get_roster_data(2023, roster_columns)
items = roster_data['status'].values
for item in items:
    print(item)

CUT
INA
INA
ACT
ACT
ACT
ACT
ACT
ACT
RES
ACT
ACT
INA
ACT
INA
RES
ACT
DEV
ACT
ACT
RET
DEV
DEV
CUT
ACT
RES
ACT
ACT
ACT
CUT
ACT
INA
ACT
ACT
ACT
ACT
ACT
ACT
ACT
ACT
ACT
ACT
ACT
CUT
ACT
ACT
ACT
ACT
ACT
ACT
ACT
ACT
RES
DEV
ACT
ACT
INA
CUT
ACT
ACT
ACT
ACT
ACT
ACT
DEV
CUT
ACT
CUT
RES
ACT
ACT
ACT
CUT
ACT
ACT
RES
ACT
ACT
ACT
ACT
RES
ACT
ACT
ACT
DEV
ACT
ACT
ACT
ACT
ACT
RES
ACT
RES
ACT
DEV
ACT
ACT
ACT
ACT
ACT
RES
ACT
ACT
ACT
INA
ACT
ACT
DEV
INA
ACT
ACT
ACT
ACT
ACT
ACT
ACT
INA
DEV
ACT
ACT
INA
ACT
ACT
ACT
ACT
ACT
ACT
ACT
DEV
CUT
ACT
ACT
ACT
RES
ACT
DEV
RES
CUT
DEV
ACT
RES
RES
ACT
ACT
ACT
ACT
CUT
RES
DEV
ACT
ACT
ACT
ACT
ACT
ACT
ACT
ACT
ACT
RES
ACT
ACT
ACT
ACT
ACT
ACT
ACT
ACT
RES
DEV
ACT
DEV
ACT
DEV
ACT
RES
ACT
CUT
RES
ACT
ACT
ACT
CUT
ACT
ACT
ACT
ACT
ACT
ACT
ACT
ACT
ACT
ACT
INA
ACT
RES
ACT
DEV
ACT
ACT
ACT
RES
RES
ACT
ACT
RES
ACT
ACT
CUT
ACT
ACT
ACT
RES
ACT
ACT
CUT
ACT
ACT
ACT
ACT
ACT
DEV
ACT
DEV
ACT
ACT
ACT
ACT
ACT
INA
ACT
DEV
ACT
DEV
ACT
ACT
ACT
DEV
ACT
ACT
ACT
DEV
ACT
ACT
DEV
CUT
ACT
ACT
CUT
ACT
ACT


In [4]:
all_players = nfl_func.get_players()
player_columns = ['status', 'display_name', 'first_name', 'last_name', 'gsis_id', 'birth_date', 'college_name', 'position_group', 'position', 'height', 'weight', 'years_of_experience', 'team_abbr', 'current_team_id', 'football_name', 'entry_year', 'rookie_year', 'draft_number', 'college_conference']
all_players = all_players[player_columns]

In [ ]:
import nfl_data_py as nfl
import pandas as pd
def get_latest_depth_chart(depth_charts):
    # Group by player and get the most recent depth chart information
    return depth_charts.sort_values('week', ascending=False).groupby(['gsis_id', 'formation', 'position']).first().reset_index()

depth_charts_2024 = nfl.import_depth_charts([2024])

# Get the latest depth chart for each player
latest_depth_charts = get_latest_depth_chart(depth_charts_2024)
latest_depth_charts

In [ ]:
import nfl_data_py as nfl
import pandas as pd
pd.set_option('display.max_rows', None)
depth_charts = nfl.import_depth_charts([2024])
depth_charts.sort_values('week', ascending=False).groupby(['gsis_id', 'formation', 'position']).first().reset_index()
depth_charts[depth_charts['club_code'] == 'CLE'].head(112)
#ftn_data.columns

In [2]:
seasons = list(range(2018, 2025))
print(seasons[len(seasons)-1])

2024


In [ ]:
weekly_data = nfl_func.get_weekly_data(2023)
weekly_data.columns

nfl_data_py contains the following functions

cache_pbp

clean_nfl_data

import_combine_data

import_contracts

import_depth_charts

import_draft_picks

import_draft_values

import_ids

import_injuries

import_ngs_data

import_officials

import_pbp_data

import_pfr

import_players

import_qbr

import_rosters

import_sc_lines

import_schedules

import_seasonal_data

import_team_desc

import_weekly_data

import_win_totals

see_pbp_cols

see_weekly_cols


In [ ]:
import pandas as pd
import numpy as np
from nfl_data_py import import_weekly_data, import_players, import_rosters, import_schedules
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
import matplotlib.pyplot as plt
import os

# 1. Fetch data
seasons = list(range(2018, 2024))  # Adjust the range as needed
player_stats = pd.concat([import_weekly_data(seasons)])
players = import_players()
latest_rosters = import_rosters([2024])  # For 2024 predictions
schedule_2024 = import_schedules([2024])

# 2. Preprocess data
df = pd.merge(player_stats, players, left_on='player_id', right_on='gsis_id', how='left')
df = df.sort_values(['player_id', 'season', 'week'])


def calculate_fanduel_fantasy_points(
    passing_yards, passing_tds, interceptions,
    rushing_yards, rushing_tds,
    receptions, receiving_yards, receiving_tds,
    fumbles, return_tds=0, two_point_conversions=0,
    field_goals_0_39=0, field_goals_40_49=0, field_goals_50_plus=0,
    extra_points=0
):
    points = 0
    
    # Passing
    points += passing_yards * 0.04
    points += passing_tds * 4
    points += interceptions * -1
    points += 3 if passing_yards >= 300 else 0
    
    # Rushing
    points += rushing_yards * 0.1
    points += rushing_tds * 6
    points += 3 if rushing_yards >= 100 else 0
    
    # Receiving
    points += receptions * 0.5
    points += receiving_yards * 0.1
    points += receiving_tds * 6
    points += 3 if receiving_yards >= 100 else 0
    
    # Misc
    points += fumbles * -2
    points += return_tds * 6
    points += two_point_conversions * 2
    
    # Kicking
    points += field_goals_0_39 * 3
    points += field_goals_40_49 * 4
    points += field_goals_50_plus * 5
    points += extra_points * 1
    
    return points

# Function to apply to DataFrame
def fanduel_fantasy_points(row):
    return calculate_fanduel_fantasy_points(
        row['passing_yards'], row['passing_tds'], row['interceptions'],
        row['rushing_yards'], row['rushing_tds'],
        row['receptions'], row['receiving_yards'], row['receiving_tds'],
        row['rushing_fumbles'] + row['receiving_fumbles'] + row.get('sack_fumbles', 0)
    )

# Create target variables
target_cols = [
                'passing_yards', 'attempts', 'completions', 'passing_tds', 'interceptions','sack_yards','sack_fumbles','sack_fumbles_lost',
                'rushing_yards', 'carries', 'rushing_tds', 'rushing_fumbles','rushing_fumbles_lost',
                'receiving_yards', 'targets', 'receptions', 'receiving_tds', 'receiving_fumbles', 'receiving_fumbles_lost', 'fanduel_fantasy_points'
            ]

# Assuming you have a DataFrame called 'df' with your data
df['fanduel_fantasy_points'] = df.apply(fanduel_fantasy_points, axis=1)

for col in target_cols:
    df[f'next_week_{col}'] = df.groupby(['player_id', 'season'])[col].shift(-1)

# Remove last week of each season for each player
df = df.groupby(['player_id', 'season']).apply(lambda x: x.iloc[:-1]).reset_index(drop=True)

columns = df.columns
for column in columns:
    print(column)

In [ ]:
from nfl_data_py import import_weekly_data, import_players, import_rosters, import_schedules, import_pbp_data
from collections import defaultdict
import pandas as pd
import ast

seasons = list(range(2018, 2024))  # Adjust the range as needed
player_stats = pd.concat([import_weekly_data(seasons)])
players = import_players()
latest_rosters = import_rosters([2024])  # For 2024 predictions
schedule_2024 = import_schedules([2024])
pbp_data = import_pbp_data(seasons)

def parse_players(player_string):
    if isinstance(player_string, str):
        return player_string.split(';')
    elif isinstance(player_string, list):
        return player_string
    else:
        return []

def calculate_player_snap_counts(pbp_data):
    snap_counts = defaultdict(lambda: defaultdict(int))
    
    for _, play in pbp_data.iterrows():
        week = play['week']
        season = play['season']
        
        offense_players = parse_players(play['offense_players'])
        defense_players = parse_players(play['defense_players'])
        
        for player in set(offense_players + defense_players):
            if player:  # Ensure we're not counting empty strings
                snap_counts[player][week] += 1
    
    snap_count_list = [
        {'player_id': player_id, 'week': week, 'season': season, 'snap_count': count}
        for player_id, weeks in snap_counts.items()
        for week, count in weeks.items()
        for week, count in weeks.items()
    ]
    
    return pd.DataFrame(snap_count_list)

# 2. Preprocess data
# Calculate snap counts for each season
snap_counts_list = []
for season in seasons:
    if season < 2021:
        weeks = list(range(1,17))
    else:
        weeks = list(range(1,18))
    for week in weeks:
        week_pbp = pbp_data[pbp_data['week'] == week]
        week_snap_counts = calculate_player_snap_counts(week_pbp)
        snap_counts_list.append(week_snap_counts)

# Combine snap counts for all seasons
all_snap_counts = pd.concat(snap_counts_list, ignore_index=True)
all_snap_counts

In [ ]:
from nfl_data_py import import_weekly_data, import_players, import_rosters, import_schedules, import_pbp_data
import pandas as pd
seasons = [2023]
pbp_data = import_pbp_data(seasons)

# Set options to display all rows and columns
pd.set_option('display.max_columns', None)
game_data = pbp_data[pbp_data['game_id'] == '2023_01_ARI_WAS']
play_type = game_data['play_type']
play_type.values

In [ ]:
# 1. Fetch data
seasons = list(range(2018, 2024))  # Adjust the range as needed
player_stats = pd.concat([import_weekly_data(seasons)])
players = import_players()

In [9]:
# Merge player stats with players data
df = pd.merge(player_stats, players, left_on='player_id', right_on='gsis_id', how='left')
df = df.sort_values(['player_id', 'season', 'week'])

In [ ]:
df

In [ ]:
import pandas as pd
from collections import defaultdict
from nfl_data_py import import_weekly_data, import_players, import_rosters, import_schedules, import_pbp_data

def parse_players(player_string):
    if isinstance(player_string, str):
        return player_string.split(';')
    elif isinstance(player_string, list):
        return player_string
    else:
        return []

import logging

logging.basicConfig(level=logging.INFO)

def categorize_play(play_type):
    if play_type in ['run', 'pass', 'no_play']:
        return 'scrimmage'
    elif play_type in ['kickoff', 'punt', 'field_goal', 'extra_point']:
        return 'special_teams'
    else:
        return 'other'

def calculate_player_snap_counts(pbp_data):
    snap_counts = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))
    team_snap_counts = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))
    
    for _, play in pbp_data.iterrows():
        game_id = play['game_id']
        posteam = play['posteam']
        defteam = play['defteam']
        play_type = play['play_type']
        
        offense_players = parse_players(play['offense_players'])
        defense_players = parse_players(play['defense_players'])
        
        # Categorize the play
        play_category = categorize_play(play_type)
        
        # Handle None values for posteam and defteam
        if posteam is None and defteam is None:
            # If both are None and play_type is None, skip this play
            if play_type is None:
                continue
            # For special teams plays, we might not have posteam/defteam info
            if play_category == 'special_teams':
                # Use a placeholder team name for special teams plays with no team info
                posteam = defteam = 'UNKNOWN_TEAM'
            else:
                continue
        elif posteam is None:
            posteam = defteam
        elif defteam is None:
            defteam = posteam
        
        # Count team snaps
        if play_category == 'special_teams':
            team_snap_counts[game_id]['special_teams'][posteam] += 1
            team_snap_counts[game_id]['special_teams'][defteam] += 1
        elif play_category == 'scrimmage':
            team_snap_counts[game_id]['offense'][posteam] += 1
            team_snap_counts[game_id]['defense'][defteam] += 1
        
        # Count player snaps
        if play_category == 'special_teams':
            for player in offense_players + defense_players:
                if player:
                    snap_counts[player][game_id]['special_teams'] += 1
        elif play_category == 'scrimmage':
            for player in offense_players:
                if player:
                    snap_counts[player][game_id]['offense'] += 1
            for player in defense_players:
                if player:
                    snap_counts[player][game_id]['defense'] += 1
    
    # Calculate percentages and create final dataframe
    snap_count_list = []
    for player_id, games in snap_counts.items():
        for game_id, counts in games.items():
            off_snaps = counts['offense']
            def_snaps = counts['defense']
            st_snaps = counts['special_teams']
            total_snaps = off_snaps + def_snaps + st_snaps
            
            # Determine the player's team for this game
            player_team = pbp_data[(pbp_data['game_id'] == game_id) & 
                                   ((pbp_data['offense_players'].apply(lambda x: player_id in x if isinstance(x, list) else player_id in str(x))) | 
                                    (pbp_data['defense_players'].apply(lambda x: player_id in x if isinstance(x, list) else player_id in str(x))))]['posteam'].iloc[0]
            
            team_off_snaps = team_snap_counts[game_id]['offense'][player_team]
            team_def_snaps = team_snap_counts[game_id]['defense'][player_team]
            team_st_snaps = team_snap_counts[game_id]['special_teams'][player_team]
            
            off_pct = (off_snaps / team_off_snaps * 100) if team_off_snaps > 0 else 0
            def_pct = (def_snaps / team_def_snaps * 100) if team_def_snaps > 0 else 0
            st_pct = (st_snaps / team_st_snaps * 100) if team_st_snaps > 0 else 0
            
            if player_id == '00-0019596':  # Tom Brady's player ID
                logging.info(f"Tom Brady - Game: {game_id}, Off snaps: {off_snaps}, Team off snaps: {team_off_snaps}, Off %: {off_pct}")
            
            snap_count_list.append({
                'player_id': player_id,
                'game_id': game_id,
                'week': pbp_data[pbp_data['game_id'] == game_id]['week'].iloc[0],
                'season': pbp_data[pbp_data['game_id'] == game_id]['season'].iloc[0],
                'offensive_snaps': off_snaps,
                'defensive_snaps': def_snaps,
                'special_teams_snaps': st_snaps,
                'total_snaps': total_snaps,
                'offensive_snap_pct': off_pct,
                'defensive_snap_pct': def_pct,
                'special_teams_snap_pct': st_pct
            })
    
    return pd.DataFrame(snap_count_list)

# 1. Fetch data
seasons = list(range(2018, 2024))  # Adjust the range as needed
player_stats = pd.concat([import_weekly_data(seasons)])
players = import_players()
latest_rosters = import_rosters([2024])  # For 2024 predictions
schedule_2024 = import_schedules([2024])
pbp_data = import_pbp_data(seasons)

# 2. Preprocess data
# Calculate snap counts for each season
snap_counts_list = []
for season in seasons:
    season_pbp = pbp_data[pbp_data['season'] == season]
    season_snap_counts = calculate_player_snap_counts(season_pbp)
    snap_counts_list.append(season_snap_counts)

# Combine snap counts for all seasons
all_snap_counts = pd.concat(snap_counts_list, ignore_index=True)

# Merge player stats with players data
df = pd.merge(player_stats, players, left_on='player_id', right_on='gsis_id', how='left')
df = df.sort_values(['player_id', 'season', 'week'])

# Merge with snap count data
df = pd.merge(df, all_snap_counts, on=['player_id', 'season', 'week'], how='left')

# Fill NaN values in snap count columns with 0
snap_columns = ['offensive_snaps', 'defensive_snaps', 'special_teams_snaps', 'total_snaps', 'offensive_snap_pct', 'defensive_snap_pct', 'special_teams_snap_pct']
for col in snap_columns:
    df[col] = df[col].fillna(0)

# If you need a single 'snap_count' column, you can create it like this:
df['snap_count'] = df['offensive_snaps'] + df['defensive_snaps'] + df['special_teams_snaps']

# Ensure all percentage columns are between 0 and 100
percentage_columns = ['offensive_snap_pct', 'defensive_snap_pct', 'special_teams_snap_pct']
for col in percentage_columns:
    df[col] = df[col].clip(0, 100)